In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam
import tensorflow as tf
# number of possible label values

from keras.applications import VGG16
# instantiate a distribution strategy

from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import layers, models, Model, optimizers
from keras.callbacks import ModelCheckpoint
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam
import tensorflow as tf
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer,GlobalAveragePooling2D
from keras.models import Sequential
import tensorflow_addons as tfa
from keras import optimizers
from keras.optimizers import SGD
import os
# number of possible label values
from keras_preprocessing.image import ImageDataGenerator


In [ ]:

trains = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')   
trains.head()   

train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')    # reading the csv file
train.head()   


In [ ]:
train["labels"]=train["labels"].apply(lambda x:x.split(" "))

In [ ]:
test_datagen=ImageDataGenerator(
    preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
          
    )
test_generator=test_datagen.flow_from_dataframe(
dataframe=trains,
directory="../input/plant-pathology-2021-fgvc8/test_images/",
x_col="image",
y_col=None,
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
save_to_dir=None,
classes=['complex','frog_eye_leaf_spot','healthy','powdery_mildew','rust','scab'],

target_size=(512,512))
#test_generator.next()

datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.10)
train_gen=test_datagen.flow_from_dataframe(
dataframe=train,
directory="../input/ppchangedimagesize/kaggle/working/train_images_224*224",
x_col="image",
y_col="labels",
     color_mode="rgb",
subset="training",
batch_size=16,
classes=['complex','frog_eye_leaf_spot','healthy','powdery_mildew','rust','scab'],
class_mode="categorical",
target_size=(512,512))

In [ ]:
model_finetuned = keras.models.load_model('../input/effpttt/weights00000014.h5')
#model_finetuned = keras.models.load_model('../input/models-for-pp/resnet_new_model_50.h5')

In [ ]:
predictions=model_finetuned.predict(test_generator)


In [ ]:
predictions

In [ ]:
class_idx=[]
for pred in predictions:
    pred=list(pred)
    temp=[]
    for i in range(0,len(pred)):
        if pred[i]>0.3:
            temp.append(i)
           
    if (temp!=[]):
        class_idx.append(temp)
    else:
        temp.append(2)
        class_idx.append(temp)
print(class_idx)

In [ ]:
#class_dict = train_gen.class_indices

In [ ]:
class_dict={'complex': 0,
 'frog_eye_leaf_spot': 1,
 'healthy': 2,
 'powdery_mildew': 3,
 'rust': 4,
 'scab': 5}

In [ ]:
def getkey(val):
    for key,value in class_dict.items():
        if (val==value):
            return key
print(class_dict)

In [ ]:
submission_pred=[]
for img in class_idx:
    img_pred=[]
    for i in img:
        print(i)
        img_pred.append(getkey(i))
    submission_pred.append( ' '.join(img_pred))

In [ ]:
results = trains[['image']]
results['labels']=submission_pred


In [ ]:
results.to_csv('submission.csv',index=False)